In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# import from website daily COVID-19 tracking for all states

- It's a good idea to view the page source first to see how it's structured
- The devtools may also be helpful
    - in Chrome right-click and choose `inspect` or just use `F12` to bring up the devtools

### Make a request using the `requests` [library](https://requests.readthedocs.io/en/master/user/quickstart/)
- `request.get()` uses http GET to get a webpage
- `request.post()` uses http POST when the webpage is submitting a form
- checking the [`status_code`](https://www.restapitutorial.com/httpstatuscodes.html) on the result let's you know your request was successful


In [2]:
website_url = 'https://docs.google.com/spreadsheets/u/2/d/e/2PACX-1vRwAqp96T9sYYq2-i7Tj0pvTf6XVHjDSMIKBdZHXiCGGdNC0ypEU9NbngS8mxea55JuCFuua1MUeOj5/pubhtml#'
response = requests.get(website_url)

response.status_code

200

### Next look at the content ot the result 
- it is a `Response` datatype
- but it looks like an html document

In [23]:
print(type(response))
response.content

<class 'requests.models.Response'>


b'<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta name="viewport" content="target-densitydpi=device-dpi,user-scalable=1,minimum-scale=1,maximum-scale=2.5,initial-scale=1,width=device-width"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><meta name="referrer" content="strict-origin-when-cross-origin"><title>Coronavirus numbers by state - Google Drive</title><link rel="shortcut icon" href="//ssl.gstatic.com/docs/spreadsheets/favicon3.ico"><link href=\'/static/spreadsheets2/client/css/2507783523-waffle_k_ltr.css\' type=\'text/css\' rel=\'stylesheet\'><style type="text/css" nonce="yCv6+O3EArcdXQv8aMjgfw">\n        html { overflow: visible; }\n        #sheets-viewport { overflow: auto; }\n        #sheets-viewport.widget-viewport { overflow: hidden; }\n        .grid-container { overflow: visible; background: white;}\n        .grid-table-container { overflow: visible; }\n        #top-bar {\n          background: url("//ssl.gstatic.com/docs/spreadsheets/publis

### The [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) package is handy for extracting data from html docs (and xml docs)

In [3]:
soup = BeautifulSoup(response.content, 'lxml')
print(soup.title)

<title>Coronavirus numbers by state - Google Drive</title>


### You can get the table that contains the data from the page using beautiful soup

In [ ]:
tables = soup.find_all('table', attrs = {'class': 'waffle'})
tables

### It is a good idea to check to see how many tables you scraped
- then use `pd.read_html()` get a list dataframes extracted from the soup tables
- you'll need to convert the tables (still a response object) to a string before pandas can read it
- to load the table you want to a dataframe, grab it from the list of dataframes

In [5]:
len(tables)

6

In [6]:
result_list = pd.read_html(str(tables[3])) # a list of dataframes
len(result_list)

1

In [7]:
covid_19_states = result_list[0]  # get the first df from the list

covid_19_states.columns = covid_19_states.iloc[0] #use first row as columns names

covid_19_states.head(10)

,1.0,Date,State,Positive,Negative,Pending,Hospitalized – Currently,Hospitalized – Cumulative,In ICU – Currently,In ICU – Cumulative,On Ventilator – Currently,On Ventilator – Cumulative,Recovered,Deaths,Data Quality Grade,Last Update ET
0,1.0,Date,State,Positive,Negative,Pending,Hospitalized – Currently,Hospitalized – Cumulative,In ICU – Currently,In ICU – Cumulative,On Ventilator – Currently,On Ventilator – Cumulative,Recovered,Deaths,Data Quality Grade,Last Update ET
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,20200604,AK,513,59584,NaN,13,NaN,NaN,NaN,1,NaN,376,10,A,6/4/2020 00:00
3,3.0,20200604,AL,19072,216227,NaN,NaN,1929,NaN,601,NaN,357,11395,653,B,6/4/2020 00:00
4,4.0,20200604,AR,8067,134413,NaN,138,757,NaN,NaN,30,127,5717,142,A,6/4/2020 00:00
5,5.0,20200604,AS,0,174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,C,6/1/2020 00:00
6,6.0,20200604,AZ,22753,227002,NaN,1079,3195,375,NaN,223,NaN,5172,996,A+,6/4/2020 00:00
7,7.0,20200604,CA,119807,2062864,NaN,4455,NaN,1279,NaN,NaN,NaN,NaN,4422,B,6/4/2020 00:00
8,8.0,20200604,CO,27060,170140,NaN,356,4443,NaN,NaN,NaN,NaN,4014,1494,A,6/4/2020 00:00
9,9.0,20200604,CT,43239,238286,NaN,373,9669,NaN,NaN,NaN,NaN,7284,4007,B,6/4/2020 16:00


In [9]:
#create dataframe from the list
df1=pd.DataFrame()
df1=df1.append(pd.DataFrame(result_list[0]))
df1.shape

(5115, 16)

In [10]:
 df1.to_csv('..\data\covid_19_daily_update_allStates.csv', index= False)

# import from website more tables on covid-19

In [11]:
df2 = pd.read_csv('https://covidtracking.com/api/v1/states/daily.csv')

In [12]:
df2.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20200604,AK,513.0,59584.0,NaN,13.0,NaN,NaN,NaN,1.0,...,60097,0,0,c1046011af7271cbe2e6698526714c6cb5b92748,0,0,0,0,0,NaN
1,20200604,AL,19072.0,216227.0,NaN,NaN,1929.0,NaN,601.0,NaN,...,235299,0,29,bcbefdb36212ba2b97b5a354f4e45bf16648ee23,0,0,0,0,0,NaN
2,20200604,AR,8067.0,134413.0,NaN,138.0,757.0,NaN,NaN,30.0,...,142480,0,26,acd3a4fbbc3dbb32138725f91e3261d683e7052a,0,0,0,0,0,NaN
3,20200604,AS,0.0,174.0,NaN,NaN,NaN,NaN,NaN,NaN,...,174,0,0,8bbc72fa42781e0549e2e4f9f4c3e7cbef14ab32,0,0,0,0,0,NaN
4,20200604,AZ,22753.0,227002.0,NaN,1079.0,3195.0,375.0,NaN,223.0,...,249755,15,66,1fa237b8204cd23701577aef6338d339daa4452e,0,0,0,0,0,NaN


In [13]:
df2.to_csv('..\data\covid_19_update_allStates_2.csv', index= False)

# import from website more columns for counties

In [14]:
df3= pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/live/us-counties.csv')

In [15]:
df3.head()

,date,county,state,fips,cases,deaths,confirmed_cases,confirmed_deaths,probable_cases,probable_deaths
0,2020-06-04,Autauga,Alabama,1001.0,242,5.0,241.0,5.0,1.0,0.0
1,2020-06-04,Baldwin,Alabama,1003.0,309,9.0,293.0,9.0,16.0,0.0
2,2020-06-04,Barbour,Alabama,1005.0,178,1.0,177.0,1.0,1.0,0.0
3,2020-06-04,Bibb,Alabama,1007.0,79,1.0,76.0,1.0,3.0,0.0
4,2020-06-04,Blount,Alabama,1009.0,65,1.0,63.0,1.0,2.0,0.0


In [16]:
df3.to_csv('..\data\covid_19_update_allStatesAndCounties_3.csv', index= False)